In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1816341,2021-11-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1816342,2021-11-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1816343,2021-11-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1816344,2021-11-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
47231,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47233,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47235,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47237,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47239,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1105294,2021-11-05,Arkansas,Arkansas,5001,3504,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1105296,2021-11-06,Arkansas,Arkansas,5001,3507,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1105298,2021-11-07,Arkansas,Arkansas,5001,3508,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1105300,2021-11-08,Arkansas,Arkansas,5001,3508,60.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1816341,2021-11-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1816342,2021-11-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1816343,2021-11-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1816344,2021-11-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-11-09') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
658,2021-11-09,Snohomish,Washington,67750,771.00,53061,WA,County,822083,8241.26,93.79
1314,2021-11-09,Cook,Illinois,649958,11902.00,17031,IL,County,5150233,12619.97,231.10
1969,2021-11-09,Orange,California,329323,5623.00,6059,CA,County,3175692,10370.12,177.06
2623,2021-11-09,Maricopa,Arizona,756169,12367.00,4013,AZ,County,4485414,16858.40,275.72
3277,2021-11-09,Los Angeles,California,1505547,26762.00,6037,CA,County,10039107,14996.82,266.58
...,...,...,...,...,...,...,...,...,...,...,...
1814897,2021-11-09,Wheeler,Oregon,108,1.00,41069,OR,County,1332,8108.11,75.08
1815290,2021-11-09,King,Texas,20,0.00,48269,TX,County,272,7352.94,0.00
1815652,2021-11-09,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10
1816010,2021-11-09,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
658,2021-11-09,Snohomish,Washington,67750,771.00,53061,WA,County,822083,8241.26,93.79,93.79,8241.26
1314,2021-11-09,Cook,Illinois,649958,11902.00,17031,IL,County,5150233,12619.97,231.10,231.10,12619.97
1969,2021-11-09,Orange,California,329323,5623.00,6059,CA,County,3175692,10370.12,177.06,177.06,10370.12
2623,2021-11-09,Maricopa,Arizona,756169,12367.00,4013,AZ,County,4485414,16858.40,275.72,275.72,16858.40
3277,2021-11-09,Los Angeles,California,1505547,26762.00,6037,CA,County,10039107,14996.82,266.58,266.58,14996.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814897,2021-11-09,Wheeler,Oregon,108,1.00,41069,OR,County,1332,8108.11,75.08,75.08,8108.11
1815290,2021-11-09,King,Texas,20,0.00,48269,TX,County,272,7352.94,0.00,0.00,7352.94
1815652,2021-11-09,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10,229.10,7101.95
1816010,2021-11-09,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1156125,2021-11-09,Hale,Alabama,3149,88.00,1065,AL,County,14651,21493.41,600.64,600.64,21493.41,1
1259295,2021-11-09,Clarke,Alabama,4813,86.00,1025,AL,County,23622,20375.07,364.07,364.07,20375.07,2
975544,2021-11-09,Winston,Alabama,4748,105.00,1133,AL,County,23629,20093.95,444.37,444.37,20093.95,3
670746,2021-11-09,Franklin,Alabama,6178,107.00,1059,AL,County,31362,19699.00,341.18,341.18,19699.00,4
326364,2021-11-09,Calhoun,Alabama,22365,511.00,1015,AL,County,113605,19686.63,449.80,449.80,19686.63,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102937,2021-11-09,Converse,Wyoming,2257,39.00,56009,WY,County,13822,16329.04,282.16,282.16,16329.04,19
972579,2021-11-09,Johnson,Wyoming,1254,16.00,56019,WY,County,8445,14849.02,189.46,189.46,14849.02,20
1373762,2021-11-09,Lincoln,Wyoming,2869,23.00,56023,WY,County,19830,14467.98,115.99,115.99,14467.98,21
1103529,2021-11-09,Sublette,Wyoming,1409,21.00,56035,WY,County,9831,14332.21,213.61,213.61,14332.21,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
903081,2021-11-09,Lowndes,Alabama,1777,67.00,1085,AL,County,9726,18270.61,688.88,688.88,18270.61,22,1
1156125,2021-11-09,Hale,Alabama,3149,88.00,1065,AL,County,14651,21493.41,600.64,600.64,21493.41,1,2
382347,2021-11-09,Walker,Alabama,11770,376.00,1127,AL,County,63521,18529.31,591.93,591.93,18529.31,16,3
799557,2021-11-09,Dallas,Alabama,5250,207.00,1047,AL,County,37196,14114.42,556.51,556.51,14114.42,62,4
901893,2021-11-09,Crenshaw,Alabama,2548,75.00,1041,AL,County,13772,18501.31,544.58,544.58,18501.31,18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724426,2021-11-09,Weston,Wyoming,1162,10.00,56045,WY,County,6927,16774.94,144.36,144.36,16774.94,16,19
1339188,2021-11-09,Uinta,Wyoming,3925,29.00,56041,WY,County,20226,19405.72,143.38,143.38,19405.72,6,20
1373762,2021-11-09,Lincoln,Wyoming,2869,23.00,56023,WY,County,19830,14467.98,115.99,115.99,14467.98,21,21
899517,2021-11-09,Albany,Wyoming,6475,36.00,56001,WY,County,38880,16653.81,92.59,92.59,16653.81,17,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,93.79,8241.26,28,28
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,93.79,8241.26,28,28
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,93.79,8241.26,28,28
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,93.79,8241.26,28,28
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,93.79,8241.26,28,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812134,2021-11-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1812135,2021-11-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1812136,2021-11-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1812137,2021-11-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1151329,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21493.41,2,1,1.00,nan
1151330,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21493.41,2,1,0.00,nan
1151331,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21493.41,2,1,0.00,nan
1151332,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21493.41,2,1,0.00,nan
1151333,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,600.64,21493.41,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397046,2021-11-05,Crook,Wyoming,926,18.00,56011,WY,County,7584,12209.92,237.34,263.71,12420.89,10,23,3.00,4.43
1397047,2021-11-06,Crook,Wyoming,926,18.00,56011,WY,County,7584,12209.92,237.34,263.71,12420.89,10,23,0.00,4.43
1397048,2021-11-07,Crook,Wyoming,926,18.00,56011,WY,County,7584,12209.92,237.34,263.71,12420.89,10,23,0.00,4.43
1397049,2021-11-08,Crook,Wyoming,936,18.00,56011,WY,County,7584,12341.77,237.34,263.71,12420.89,10,23,10.00,4.43


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
899465,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18270.61,1,22,1.00,3.57,0.00,nan
899466,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18270.61,1,22,0.00,3.00,0.00,nan
899467,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18270.61,1,22,0.00,2.86,0.00,nan
899468,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18270.61,1,22,0.00,2.86,0.00,nan
899469,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18270.61,1,22,0.00,2.57,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378112,2021-11-05,Teton,Wyoming,5243,14.00,56039,WY,County,23464,22344.87,59.67,59.67,22506.82,23,3,2.00,8.86,0.00,0.00
378113,2021-11-06,Teton,Wyoming,5243,14.00,56039,WY,County,23464,22344.87,59.67,59.67,22506.82,23,3,0.00,8.86,0.00,0.00
378114,2021-11-07,Teton,Wyoming,5243,14.00,56039,WY,County,23464,22344.87,59.67,59.67,22506.82,23,3,0.00,8.86,0.00,0.00
378115,2021-11-08,Teton,Wyoming,5269,14.00,56039,WY,County,23464,22455.68,59.67,59.67,22506.82,23,3,26.00,8.36,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-11-09') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
456839,2021-11-09,Imperial,California,37040,765.00,6025,CA,County,181215,20439.81,422.15,422.15,20439.81,1,3,16.00,55.57,0.00,0.00
226715,2021-11-09,San Bernardino,California,366108,5836.00,6071,CA,County,2180085,16793.29,267.70,267.70,16793.29,2,6,253.00,431.21,4.00,5.86
3277,2021-11-09,Los Angeles,California,1505547,26762.00,6037,CA,County,10039107,14996.82,266.58,266.58,14996.82,3,14,1179.00,1302.07,10.00,13.07
101013,2021-11-09,Stanislaus,California,89331,1380.00,6099,CA,County,550660,16222.53,250.61,250.61,16222.53,4,7,109.00,175.50,2.00,2.79
1104050,2021-11-09,Tuolumne,California,7539,134.00,6109,CA,County,54478,13838.61,245.97,245.97,13838.61,5,20,21.00,25.79,3.00,1.14
80275,2021-11-09,San Joaquin,California,105221,1798.00,6077,CA,County,762148,13805.85,235.91,235.91,13805.85,6,21,40.00,136.07,4.00,3.79
726809,2021-11-09,Merced,California,44342,644.00,6047,CA,County,277680,15968.74,231.92,231.92,15968.74,7,9,26.00,83.93,36.00,4.00
802484,2021-11-09,Inyo,California,2436,41.00,6027,CA,County,18039,13504.07,227.29,227.29,13504.07,8,23,4.00,15.64,0.00,0.00
974300,2021-11-09,Kings,California,34334,344.00,6031,CA,County,152940,22449.33,224.92,224.92,22449.33,9,2,27.00,72.07,0.00,0.64
69889,2021-11-09,Shasta,California,25229,395.00,6089,CA,County,180080,14009.88,219.35,219.35,14009.88,10,18,25.00,73.21,4.00,2.93


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1712212,2021-11-09,Lassen,California,7812,53.00,6035,CA,County,30573,25551.96,173.36,173.36,25551.96,20,1,64.00,19.21,1.00,0.21
974300,2021-11-09,Kings,California,34334,344.00,6031,CA,County,152940,22449.33,224.92,224.92,22449.33,9,2,27.00,72.07,0.00,0.64
456839,2021-11-09,Imperial,California,37040,765.00,6025,CA,County,181215,20439.81,422.15,422.15,20439.81,1,3,16.00,55.57,0.00,0.00
125370,2021-11-09,Tulare,California,83872,1020.00,6107,CA,County,466195,17990.75,218.79,218.79,17990.75,11,4,54.00,194.64,0.00,2.43
285345,2021-11-09,Kern,California,152732,1706.00,6029,CA,County,900202,16966.41,189.51,189.51,16966.41,14,5,314.00,353.43,0.00,3.86
226715,2021-11-09,San Bernardino,California,366108,5836.00,6071,CA,County,2180085,16793.29,267.70,267.70,16793.29,2,6,253.00,431.21,4.00,5.86
101013,2021-11-09,Stanislaus,California,89331,1380.00,6099,CA,County,550660,16222.53,250.61,250.61,16222.53,4,7,109.00,175.50,2.00,2.79
46004,2021-11-09,Madera,California,25281,291.00,6039,CA,County,157327,16069.08,184.97,184.97,16069.08,17,8,28.00,75.93,0.00,0.43
726809,2021-11-09,Merced,California,44342,644.00,6047,CA,County,277680,15968.74,231.92,231.92,15968.74,7,9,26.00,83.93,36.00,4.00
1041305,2021-11-09,Glenn,California,4502,46.00,6021,CA,County,28393,15856.02,162.01,162.01,15856.02,23,10,3.00,6.86,0.00,0.29


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
456839,2021-11-09,Imperial,California,37040,765.00,6025,CA,County,181215,20439.81,422.15,422.15,20439.81,1,3,16.00,55.57,0.00,0.00
226715,2021-11-09,San Bernardino,California,366108,5836.00,6071,CA,County,2180085,16793.29,267.70,267.70,16793.29,2,6,253.00,431.21,4.00,5.86
3277,2021-11-09,Los Angeles,California,1505547,26762.00,6037,CA,County,10039107,14996.82,266.58,266.58,14996.82,3,14,1179.00,1302.07,10.00,13.07
101013,2021-11-09,Stanislaus,California,89331,1380.00,6099,CA,County,550660,16222.53,250.61,250.61,16222.53,4,7,109.00,175.50,2.00,2.79
1104050,2021-11-09,Tuolumne,California,7539,134.00,6109,CA,County,54478,13838.61,245.97,245.97,13838.61,5,20,21.00,25.79,3.00,1.14
80275,2021-11-09,San Joaquin,California,105221,1798.00,6077,CA,County,762148,13805.85,235.91,235.91,13805.85,6,21,40.00,136.07,4.00,3.79
726809,2021-11-09,Merced,California,44342,644.00,6047,CA,County,277680,15968.74,231.92,231.92,15968.74,7,9,26.00,83.93,36.00,4.00
802484,2021-11-09,Inyo,California,2436,41.00,6027,CA,County,18039,13504.07,227.29,227.29,13504.07,8,23,4.00,15.64,0.00,0.00
974300,2021-11-09,Kings,California,34334,344.00,6031,CA,County,152940,22449.33,224.92,224.92,22449.33,9,2,27.00,72.07,0.00,0.64
69889,2021-11-09,Shasta,California,25229,395.00,6089,CA,County,180080,14009.88,219.35,219.35,14009.88,10,18,25.00,73.21,4.00,2.93


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,68.57
1,03/21/20,Imperial,4,2.21,3,0.00,60.93
2,03/22/20,Imperial,4,2.21,3,0.00,50.86
3,03/23/20,Imperial,4,2.21,3,0.00,50.86
4,03/24/20,Imperial,9,4.97,3,5.00,51.21
...,...,...,...,...,...,...,...
5950,11/05/21,Glenn,4485,15796.15,10,9.00,7.21
5951,11/06/21,Glenn,4485,15796.15,10,0.00,7.21
5952,11/07/21,Glenn,4485,15796.15,10,0.00,7.21
5953,11/08/21,Glenn,4499,15845.45,10,14.00,8.21


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6059,11/05/21,Shasta,384.00,213.24,10,7.00,2.29
6060,11/06/21,Shasta,384.00,213.24,10,0.00,2.29
6061,11/07/21,Shasta,384.00,213.24,10,0.00,2.29
6062,11/08/21,Shasta,391.00,217.13,10,7.00,2.64


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)